In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('../datasets/primary_dataset.csv')
df_shopping_place = pd.read_csv('../datasets/jumlah_mart_bandung.csv')

df_shopping_place_count = df_shopping_place['jumlah']

# Clean and standardize kecamatan names in both dataframes
df['kecamatan'] = df['kecamatan'].str.lower().str.strip()
df_shopping_place['kecamatan'] = df_shopping_place['kecamatan'].str.lower().str.strip()

# Fix specific mismatches
df.loc[df['kecamatan'] == 'ujungberung', 'kecamatan'] = 'ujung berung'
df_shopping_place.loc[df_shopping_place['kecamatan'] == 'ujungberung'] = 'ujung berung'

# Create mapping from df_shopping_place
shopping_mapping = df_shopping_place.groupby('kecamatan')['jumlah'].sum().to_dict()

# Map the shopping place counts to df
df['jumlah_shopping_place'] = df['kecamatan'].map(shopping_mapping)

# Check for any unmapped values
unmapped = df[df['jumlah_shopping_place'].isna()]['kecamatan'].unique()

# Create fnb_count by counting restaurants per kecamatan
fnb_count = df[df['type'] == 'restaurant'].groupby('kecamatan').size().reset_index(name='jumlah_restaurant')

# Create mapping from fnb_count and map to df
fnb_mapping = fnb_count.set_index('kecamatan')['jumlah_restaurant'].to_dict()
df['fnb_count'] = df['kecamatan'].map(fnb_mapping)
# Display the fnb_count

# Fill missing demographic data for rows with same kecamatan
kecamatan_demo_mapping = df.groupby('kecamatan').agg({
    'Jumlah Penduduk': 'first',
    'Luas Wilayah (km²)': 'first', 
    'Kepadatan (jiwa/km²)': 'first'
}).to_dict('index')

# Track which kecamatan couldn't be filled
failed_kecamatan = set()

for idx, row in df.iterrows():
    kecamatan = row['kecamatan']
    
    # Check if any demographic data is missing
    if pd.isna(row['Jumlah Penduduk']) or pd.isna(row['Luas Wilayah (km²)']) or pd.isna(row['Kepadatan (jiwa/km²)']):
        if kecamatan in kecamatan_demo_mapping:
            # Fill missing values from mapping
            if pd.isna(row['Jumlah Penduduk']):
                df.at[idx, 'Jumlah Penduduk'] = kecamatan_demo_mapping[kecamatan]['Jumlah Penduduk']
            if pd.isna(row['Luas Wilayah (km²)']):
                df.at[idx, 'Luas Wilayah (km²)'] = kecamatan_demo_mapping[kecamatan]['Luas Wilayah (km²)']
            if pd.isna(row['Kepadatan (jiwa/km²)']):
                df.at[idx, 'Kepadatan (jiwa/km²)'] = kecamatan_demo_mapping[kecamatan]['Kepadatan (jiwa/km²)']
        else:
            failed_kecamatan.add(kecamatan)

# Print kecamatan that couldn't be filled
if failed_kecamatan:
    print("Kecamatan yang tidak ditemukan atau gagal diisi:")
    for kec in failed_kecamatan:
        print(f"- {kec}")
else:
    print("Semua data kecamatan berhasil diisi")

print(f"\nJumlah fnb_count: {fnb_count.shape[0]}")
print(fnb_count.head())


#Ensure theres no duplicates in the df
df.drop_duplicates(inplace=True)

df.dropna(inplace=True)
df.isnull().sum()


# df_null.isna().sum()

# # Create heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(df.select_dtypes(include=np.number), annot=True, cmap='coolwarm')
# plt.title('Correlation Matrix Heatmap')
# plt.tight_layout()
# plt.show()


Semua data kecamatan berhasil diisi

Jumlah fnb_count: 31
         kecamatan  jumlah_restaurant
0            andir                188
1         antapani                 95
2        arcamanik                 60
3      astanaanyar                 73
4  babakan ciparay                 86


Unnamed: 0               0
nama                     0
alamat                   0
kecamatan                0
type                     0
tahun                    0
source_file              0
Jumlah Penduduk          0
Luas Wilayah (km²)       0
Kepadatan (jiwa/km²)     0
jumlah_shopping_place    0
fnb_count                0
dtype: int64